In [1]:
from urllib import request
import os
import re
import pandas as pd
import csv
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/Shared drives/"

# choose where you want your project files to be saved
project_folder = "MS - Translation/Sentence Alignment/vecalign_embeddings"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

create_and_set_working_directory(project_folder)

In [26]:
!ls

corpus_test  get_labse_embeddings.ipynb


In [5]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [6]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(sentences)
print(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

[[ 0.02882476 -0.00602381 -0.05947006 ... -0.0300225  -0.02960703
   0.00067479]
 [-0.05550232  0.02546487 -0.02157256 ...  0.02932104  0.01150042
  -0.00848786]]


In [8]:
len(embeddings[1])

768

In [9]:
type(embeddings[1])

numpy.ndarray

In [32]:
def embed_text_file(input_file, output_file):
    # Load the LaBSE model
    model = SentenceTransformer('sentence-transformers/LaBSE')

    # Read the text file and split into sentences
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = text.split('\n')  # Assuming each sentence is on a new line

    # Compute the embeddings
    embeddings = model.encode(sentences)

    # Convert the embeddings to float32 numpy array
    embeddings_np = np.array(embeddings, dtype=np.float32)

    # Save the embeddings as a binary file
    embeddings_np.tofile(output_file)

# Usage example
input_file = 'corpus_test/tur/chapter1_sentences_overlaps.txt'  # The text file you want to process
output_file = 'corpus_test/tur/chapter1_sentences_overlaps.emb'  # The output binary file
embed_text_file(input_file, output_file)

vecalign wants embeddings size 1024 but default labse is 768

In [30]:
import torch
from torch import nn
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the LaBSE model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Define a projection layer
class ProjectionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Initialize the projection layer (from 768 to 1024)
projection_layer = ProjectionLayer(768, 1024)

def embed_and_project_text_file(input_file, output_file):
    # Read the text file and split into sentences
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = text.split('\n')  # Assuming each sentence is on a new line

    # Compute the embeddings
    embeddings = model.encode(sentences)

    # Convert embeddings to torch tensor
    embeddings_tensor = torch.tensor(embeddings)

    # Project the embeddings to the desired dimension
    projected_embeddings = projection_layer(embeddings_tensor)

    # Convert the projected embeddings to float32 numpy array
    projected_embeddings_np = projected_embeddings.detach().numpy().astype(np.float32)

    # Ensure the final embedding size is 1024
    assert projected_embeddings_np.shape[1] == 1024, f"Expected embedding size of 1024, got {projected_embeddings_np.shape[1]}"

    # Save the projected embeddings as a binary file
    projected_embeddings_np.tofile(output_file)

# Usage example
input_file = 'corpus_test/eng/chapter1_sentences_overlaps.txt'  # The text file you want to process
output_file = 'corpus_test/eng/chapter1_sentences_overlaps.emb'  # The output binary file
embed_and_project_text_file(input_file, output_file)


maybe try laser??

In [14]:
!pip install laser_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.1/895.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 29.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291755 sha256=48c129b20b87201a570c132cd7ad25b2bb2bd0b508c905508b481e9700e76e5e
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b95

In [27]:
from laser_encoders import LaserEncoderPipeline
import numpy as np

def encode_text_file(input_file, output_file):
    # Initialize the LASER encoder
    encoder = LaserEncoderPipeline(lang="eng_Latn")

    # Read the text file and split into sentences
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = text.split('\n')  # Assuming each sentence is on a new line

    # Compute the embeddings
    embeddings = encoder.encode_sentences(sentences)

    # Ensure the embeddings are in float32 format
    embeddings_np = np.array(embeddings, dtype=np.float32)

    # Save the embeddings as a binary file
    embeddings_np.tofile(output_file)

# Usage example
input_file = 'corpus_test/eng/chapter1_sentences_overlaps.txt'  # The text file you want to process
output_file = 'corpus_test/eng/chapter1_sentences_overlaps.emb'  # The output binary file
encode_text_file(input_file, output_file)


In [28]:
from laser_encoders import LaserEncoderPipeline
import numpy as np

def encode_text_file(input_file, output_file):
    # Initialize the LASER encoder
    encoder = LaserEncoderPipeline(lang="tur_Latn")

    # Read the text file and split into sentences
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = text.split('\n')  # Assuming each sentence is on a new line

    # Compute the embeddings
    embeddings = encoder.encode_sentences(sentences)

    # Ensure the embeddings are in float32 format
    embeddings_np = np.array(embeddings, dtype=np.float32)

    # Save the embeddings as a binary file
    embeddings_np.tofile(output_file)

# Usage example
input_file = 'corpus_test/tur/chapter1_sentences_overlaps.txt'  # The text file you want to process
output_file = 'corpus_test/tur/chapter1_sentences_overlaps.emb'  # The output binary file
encode_text_file(input_file, output_file)
